# Working With Algorithms

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/main?filepath=docs%2Fexamples%2F%2Fnotebooks%2FWorking_with_Algorithm.ipynb)

For this demo we will use some elevation data from https://www.swisstopo.admin.ch/fr/geodata/height/alti3d.html  dataset

#### Requirements
- folium
- httpx

`!pip install folium httpx`

In [ ]:
import json
import httpx

from folium import Map, TileLayer

In [ ]:
titiler_endpoint = "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.

In [ ]:
url = "https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2573-1085/swissalti3d_2019_2573-1085_0.5_2056_5728.tif"

## Get COG Info

In [ ]:
# Fetch dataset Metadata
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

print(r)

### Display Tiles

By default, the tiles will be rescaled from min/max from dataset statistics (1615.812 / 2015.09448)

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Office fédéral de topographie swisstopo"
).add_to(m)
m

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        # rio-tiler cannot rescale automatically the data when using a colormap
        "rescale": "1615.812,2015.09448",
        "colormap_name": "terrain",
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Office fédéral de topographie swisstopo"
)
aod_layer.add_to(m)
m

## Show Available Algorithms

In [ ]:
# Fetch algorithms
print("Available algorithm")
print(list(httpx.get(f"{titiler_endpoint}/algorithms").json()))
print()
print("Metadata from `Hillshade` algorithm")
meta = httpx.get(f"{titiler_endpoint}/algorithms/hillshade").json()
print("Inputs")
print(meta["inputs"])
print("Outputs")
print(meta["outputs"])
print("Parameters")
print(meta["parameters"])

## Display Hillshade Tiles

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        "algorithm": "hillshade",
        # Hillshade algorithm use a 3pixel buffer so we need
        # to tell the tiler to apply a 3 pixel buffer around each tile
        "buffer": 3,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
)
aod_layer.add_to(m)
m

### Pass parameters to the algorithm

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        "algorithm": "contours",
        "algorithm_params": json.dumps(
            {
                "increment": 20, # contour line every 20 meters
                "thickness": 2, # 2m thickness
                "minz": 1600,
                "maxz": 2000
            }
        ),
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
).add_to(m)
m

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        "algorithm": "contours",
        "algorithm_params": json.dumps(
            {
                "increment": 5, # contour line every 5 meters
                "thickness": 1, # 1m thickness
                "minz": 1600,
                "maxz": 2000
            }
        ),
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
).add_to(m)
m